sgdclassifier | bag of sites | 0.93362

csr_matrix заменён на CountVectorizer; local провек

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score

PATH_TO_DATA = '../capstone_user_identification'

In [2]:
# !unzip capstone_user_identification.zip

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [4]:
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [6]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [7]:
train_test_df_sites

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
...,...,...,...,...,...,...,...,...,...,...
82793,812,1039,676,0,0,0,0,0,0,0
82794,300,302,302,300,300,1222,302,1218,1221,1216
82795,29,33,35,22,37,6779,30,21,23,6780


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
with open('tmp/train.csv') as train:
    X_train_sparse = cv.fit_transform(train)
with open('tmp/test.csv') as test:
    X_test_sparse = cv.transform(test)
y = train_df['target']

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=17, n_jobs=-1)

In [12]:
%%time
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(lr, X_train_sparse, y, cv=5, scoring='roc_auc')

CPU times: user 370 ms, sys: 492 ms, total: 862 ms
Wall time: 12.1 s


In [13]:
cv_scores, cv_scores.mean()

(array([0.96461068, 0.96232129, 0.96437419, 0.96028937, 0.9619478 ]),
 0.9627086649832515)

**-----**

In [17]:
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [18]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [19]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'spec_submission1.csv') # leaderboard 0.91646

Бейзлайн sgd_logit_benchmark: 0.91273 побит